# **AAHPS: ASSIGNMENT 5**

**Authors**: Nina Mislej and Nika Molan
<br/>**Student numbers**: 63200016 and 63200017

### **Exercise Description**
The aim of this assignment is to find the best result for **24 optimization** (*minimization*) **functions** that are available as **BBOB** (*Black-Box Optimization Benchmark*). We will implement **4 optimization programs** one of which has to be **local search** and the other ones can implement **any** optimization approach.
<br/>The functions are available in ***smoof*** package in **R** and we will initialize the functions with **40 dimensions** using the *iid*: **2023**

In addition to this report the results include the **coordinates** for each of the **24 minimums**. These are included in a separate file, one for each algorithm. 
<br/>One line represents **40**-**touple** for one function

### **Functions Description**

The **[functions](https://numbbo.github.io/gforge/downloads/download16.00/bbobdocfunctions.pdf)** are designed to cover a lot of different problems that occur with **different optimization approaches**. Some of the functions are meant to test if the algorithm gets **stuck in local optimums**, some check if the algorithm can find optimums **bordering** the function domain, some functions are symetric while some highly asymetric and so on and so forth. We have to take this properties in to account when chosing our algorithms to optimize the search.

In [389]:
# setting up the package and enviroment
import numpy as np
import math
import pandas as pd
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

numpy2ri.activate() # automatic conversion from numpy to R arrays
smoof = importr("smoof") # importing R smoof package

Now we **initialize** the functions.
</br>In this process we also save their known minimums, which are provided in their description. We will use these minimums later on **to compare** the coordinates we get from our optimization algorithms.

In [175]:
# initializing the functions and checking their minimums
functions = {}
true_minimums = {}

for fun in range(1, 25):
    functions[fun] = (smoof.makeBBOBFunction(40, fun, 2023))
    true_minimums[fun] = smoof.getGlobalOptimum(functions[fun]).rx2("value")[0]
    print(f"MINIMUM OF FUNCTION {fun}: {true_minimums[fun]}")

# setting the bounds:
upper_bound = 5
lower_bound = -5

MINIMUM OF FUNCTION 1: 21.1
MINIMUM OF FUNCTION 2: 26.91
MINIMUM OF FUNCTION 3: 311.6
MINIMUM OF FUNCTION 4: 311.6
MINIMUM OF FUNCTION 5: -48.47
MINIMUM OF FUNCTION 6: -91.36
MINIMUM OF FUNCTION 7: 32.49
MINIMUM OF FUNCTION 8: 71.6
MINIMUM OF FUNCTION 9: -356.7
MINIMUM OF FUNCTION 10: 51.03
MINIMUM OF FUNCTION 11: -96.65
MINIMUM OF FUNCTION 12: 553.39
MINIMUM OF FUNCTION 13: 9.88
MINIMUM OF FUNCTION 14: 405.47
MINIMUM OF FUNCTION 15: 64.25
MINIMUM OF FUNCTION 16: -43.28
MINIMUM OF FUNCTION 17: 227.51
MINIMUM OF FUNCTION 18: 227.51
MINIMUM OF FUNCTION 19: 73.06
MINIMUM OF FUNCTION 20: -123.81
MINIMUM OF FUNCTION 21: -44.42
MINIMUM OF FUNCTION 22: 222.1
MINIMUM OF FUNCTION 23: -1000.0
MINIMUM OF FUNCTION 24: -1.33


In [176]:
# comparing minimums we got with the real ones
def comparison(approx_minimums):
    error = 0
    for i,(true,approx) in enumerate(zip(true_minimums.values(), approx_minimums),1):
        print(f"{i : <20} \
              TRUE: {true : <20} \
              APPROXIMATE: {round(approx,2) : <20} \
              DIFFERENCE: {round(abs(true - approx),2)}")
        
        error = error + round(abs(true - approx),2)
    print(f"OVERALL ABSOLUTE ERROR:  {error}")

# writing the coordinates of the minimum to file
def results_to_file(coordinates, algo_number):
    with open(f"algorithm_{algo_number}.txt", 'w') as f:
        for point in coordinates:
            for xi in point:
                f.write(f"{xi} ")
            f.write("\n")

## **ALGORITHM 1:** Gradient Descent

This algorithm was chosen as the **first one** we tested, under the assumtion it is often wildly used and works for functions that behave well. One big problem is the differentiability of the functions which is not always favourable and is sometimes time consuming so we used an **approximation**. 15 out of the 17 functions with description are described as **differentiable** so it is worth analysing this approach, even though it is a bit naive because it only performs well for smooth, convex, unimodal functions. The problem with this aproach is also that it has a big possibility of getting stuck in a **local minimum**. We will be solving this problem by adding a random component to it, making the algorithm go through **more iterations** starting at **different points** chosen uniformly at random.

The idea is that we start in a randomly chosen point and compute the **gradient** for that set of coordinates. The function **``gradient_approximation``** takes care of that. Instead of differentiating each variable, we use two points that are 0.01 apart. Because this is the direction of the **biggest increase** for the function we are trying to optimize we compute the next point by going in the opposite direction of the mentioned increase. This is the equation $x_n = x_{n-1} - \alpha \cdot \bigtriangleup G(x_n)$ where $\bigtriangleup G$ is the gradient of the objective function.

The next question that arises in this proposition is what is the **step size** in this direction? Let's look at the **``alpha``** parameter in the code bellow. Now there are many ways to find this scalar, the optimal one would be finding the one that **minimizes** the equation: $G(x_{n-1} - \alpha \cdot \bigtriangleup G(x_n))$. This would take a lot of time because we are working with 40 dimentions and after some observations one can notice the elements of the gradients sometimes tend to be very different in size. We tackled with this problem by making the step size for each partial derivative different making it **between 0 and 1** for every single one. This also solves a lot of domain breach problems. We wrapped all of this up in a function that creates the new point **``new_point``**.

***NOTE***: This algorithm is quite **time consuming** for all 24 functions, because we have to calculate the **gradient approximation**.

In [421]:
# function implementation
def gradient_descent(limit, dim, fun):
    
    # setting the best solution to the first one we try and updating it later on 
    # this is not neccessary and could be omited 
    # but given our lambda calculation we could skip to a worse solution
    best_coordinates = np.random.uniform(lower_bound, upper_bound, dim)
    best_value = fun(best_coordinates)[0]

    # our solution in each step - starting position
    curr_coordinates = best_coordinates.copy()

    # limit specifies the number of descents
    
    for k in range(limit):
        gradient = gradient_approximation(fun, curr_coordinates, dim)
        if gradient == -1: break

        curr_coordinates = new_point(curr_coordinates, gradient)
        curr_value = fun(curr_coordinates)[0]

        if curr_value < best_value:
            best_value = curr_value
            best_coordinates = curr_coordinates.copy()
            

    return best_coordinates, best_value
           
# approximates all partial derivatives in the gradient            
def gradient_approximation(function, x0, dim):
    approx = []
    x1 = x0.copy()

    # approximation for each variable
    for i in range(dim):
        x1[i] = x0[i] + 0.01
        y0 = function(x0)
        y1 = function(x1)

        # if the difference is too small we can 
        # increase the difference to move the point still
        if math.isnan(y1 - y0): x1[i] = x1[i] + 0.04
        approx.append(((y1 - y0)/(x1[i] - x0[i]))[0])
        x1[i] = x0[i]
    return approx

# calculating the lambda and deciding on a new point
def new_point(point, gradient):
    new = []
    for xi, grad in zip(point, gradient):

        # calculating te number of digits in each derivative
        no_digits = len(str(abs(math.floor(grad))))
        alpha = 10**(-no_digits)
        next_point = xi - alpha * grad

        # checking the constraints
        if next_point < upper_bound and next_point > lower_bound: 
            new.append(next_point)
        else: new.append(xi)
    return new

In [167]:
# testing our algorithm
min_values_1 = [float('inf') for i in range(len(functions))]
min_coordinates_1 = [[0 for i in range(40)] for k in range(len(functions))]
no_runs = 100

for i in range(no_runs):
    for k in range(len(functions)):
        curr_coordinates, curr_value = \
            gradient_descent(limit=300, dim=40, fun=functions[k+1])
        if curr_value < min_values_1[k]:
            min_values_1[k] = curr_value
            min_coordinates_1[k] = curr_coordinates.copy()
    
comparison(min_values_1)
results_to_file(min_coordinates_1, 1)

1                    TRUE: 21.1                 APPROXIMATE: 21.1                 DIFFERENCE: 0.0
2                    TRUE: 26.91                APPROXIMATE: 39815.63             DIFFERENCE: 39788.72
3                    TRUE: 311.6                APPROXIMATE: 1441.91              DIFFERENCE: 1130.31
4                    TRUE: 311.6                APPROXIMATE: 1747.75              DIFFERENCE: 1436.15
5                    TRUE: -48.47               APPROXIMATE: -23.91               DIFFERENCE: 24.56
6                    TRUE: -91.36               APPROXIMATE: -29.92               DIFFERENCE: 61.44
7                    TRUE: 32.49                APPROXIMATE: 2225.85              DIFFERENCE: 2193.36
8                    TRUE: 71.6                 APPROXIMATE: 499.89               DIFFERENCE: 428.29
9                    TRUE: -356.7               APPROXIMATE: -120.22              DIFFERENCE: 236.48
10                   TRUE: 51.03                APPROXIMATE: 139871.12            DIFFERENC

Taking these results in to consideration we could deffenitly do better. We can see that in general this algorithm **does not perform well time-wise**, at least not given these parameters.
While it does produce quite good results for some fuctions, we can see it fails most drastically in the case of **function 2**, **10** and **12**. Let us try a **local search** approach next to see if we could perhaps imporve these results.

## **ALGORITHM 2:** Simulated Annealing

This was the **second one** algorithm. The reason behind this one was the fact that these functions are very different, so the idea of some improved **random search** could work if we are trying to get the best possible result for all functions even though this could mean cutting some loses at those more specific ones. 

Now for a quick summery of how this algorithm works. We start with random points for each function. At every iteration of the algorithm take a **random step** between -0.1 and 0.1 for all 40 dimensions and check whether this **neighbour** produces a **better objective value**. Now in order not to get stuck in a **local minimum** we decide to move to this new point even if the solution is **worse** with some **probability** that is getting smaller with each iteration. This probability is regulated with a parameter called **``temperature``**. The higher the temperature the bigger the probability of accepting the bad solution, the more search space we explore. The lower the temperature, the more we focus on the solution at hand and optimizing this one. 

In our code **``temp``** is the initial temperature which is used to calculate the actual temperature in each step of the loop. The condition that decides the probability is taken from the Metropolis algorithm and goes as follows: 
1. first we calculate the difference between the current value and the candidate value of the neighbour: $\bigtriangleup$
2. if the difference is a negative one then the solution is better and we accept and save it - this is now our current position
3. otherwise we accept the probability with the Boltzmann distribution: $P = e^{\cfrac {\bigtriangleup} {temperature}}$
4. we generate a random number and if the number is higher we move to the candidate point

In [426]:
# implementation
def simulated_annealing(limit, dim, temp, fun):
    
    # setting the best solution to the first one we try and updating it later on
    best_coordinates = np.random.uniform(lower_bound, upper_bound, dim)
    best_value = fun(best_coordinates)[0]

    # our solution in each step - starting position
    curr_coordinates = best_coordinates.copy()
    curr_value = best_value
    
    
    for k in range(limit):

        # calculating the neighbour candidate and checking the bounds
        candidate = curr_coordinates + np.random.uniform(-0.1, 0.1, 40)
        if all(xi < upper_bound and xi > lower_bound for xi in candidate):
            
            # if the value is better than the best one so far we save it
            candidate_value = fun(candidate)[0]
            if candidate_value < best_value:
                best_coordinates = candidate.copy()
                best_value = candidate_value
            
            # calculating the difference between the values and the new temperature
            diff = candidate_value - curr_value
            temperature = temp / float(k + 1)

            # calculating the probability of accepting a bad solution
            x = -diff / temperature
            try: condition = math.exp(x)
            except OverflowError: condition = float("Inf")

            # random throw whether we accept the solution
            if diff < 0 or np.random.random() < condition:
                curr_coordinates = candidate.copy()
                curr_value = candidate_value
            
    return [best_coordinates, best_value]

In [168]:
# testing our algorithm
min_values_2 = [float('inf') for i in range(len(functions))]
min_coordinates_2 = [[0 for i in range(40)] for k in range(len(functions))]
no_runs = 300

for i in range(no_runs):
    for k in range(len(functions)):
        curr_coordinates, curr_value =\
            simulated_annealing(limit=3000, dim=40, temp=100, fun=functions[k+1])
        if curr_value < min_values_2[k]:
            min_values_2[k] = curr_value
            min_coordinates_2[k] = curr_coordinates.copy()
            

comparison(min_values_2)
results_to_file(min_coordinates_2, 2)

1                    TRUE: 21.1                 APPROXIMATE: 21.39                DIFFERENCE: 0.29
2                    TRUE: 26.91                APPROXIMATE: 25920.63             DIFFERENCE: 25893.72
3                    TRUE: 311.6                APPROXIMATE: 1256.07              DIFFERENCE: 944.47
4                    TRUE: 311.6                APPROXIMATE: 1330.9               DIFFERENCE: 1019.3
5                    TRUE: -48.47               APPROXIMATE: 168.67               DIFFERENCE: 217.14
6                    TRUE: -91.36               APPROXIMATE: -67.81               DIFFERENCE: 23.55
7                    TRUE: 32.49                APPROXIMATE: 433.45               DIFFERENCE: 400.96
8                    TRUE: 71.6                 APPROXIMATE: 122.27               DIFFERENCE: 50.67
9                    TRUE: -356.7               APPROXIMATE: -310.42              DIFFERENCE: 46.28
10                   TRUE: 51.03                APPROXIMATE: 26798.82             DIFFERENCE: 

This result is **overall better**, if we take the absolute error as a measurment of succsess, but gradient descent performed better for a few **specific cases**. The difference in the result is mainly due to different results in the case of **function 12** and **10**. These were two simple algorithms, now on to a pair of more complicated ones.

## **ALGORITHM 3:** Covariance Matrix Adaptation Evolution Strategy
Considering none of the algorithms come close to all of the actual minimums, we decided to look for any **academic articales** on these functions and found the paper titled 
</br>**[Comparing Results of 31 Algorithms from the Black-Box Optimization Benchmarking BBOB](https://www.researchgate.net/publication/220740712_Comparing_Results_of_31_Algorithms_from_the_Black-Box_Optimization_Benchmarking_BBOB-2009#fullTextFileContent)**. It seemed that the best result for functions in **40 dimensions** was the **BI-population CMA-ES algorithm**, so we looked it up and decided to implement our own version of a **[CMA-ES](https://arxiv.org/pdf/1604.00772.pdf)** algorithm. This is an **evolutionary algorithm** which is an approach we have not yet tried, so perhaps this will lead to a better solution. It's basis relies on well known evolutionary concepts, mainly the importance of mutation for providing more **variety** to the search and **selection** where only the better offsprings remain.

The actual implementation of the algorithm is quite complicated but let us try and recap the important details and the main idea behind this strategy. In the CMA Evolution Strategy, a population of new search points often called offsprings is generated by **sampling** a **multivariate normal distribution**, which is the generalization of the normal distribution in higher dimensions. It is uniquely determined by the **mean** **$m$** and the positive definite **covariance matrix $C$**. The **``sigma``** in our code again represents the step size in this sampling method.

This would be the basic formula for finding **next generation offsprings**:
${x_k^{(g+1)}} = m^{(g)} + {\sigma}^{(g)} \cdot N(0, C^{(g)})$
- $C$ is the covariance matrix at generation $g$
- $m$ is the mean value of the search distribution at generation $g$
- $\sigma$ is the overall standard deviation represented with our **``sigma``**
- $k$ is in the range of $\lambda$ of our population size, the number of offsprings: **``offsprings``**
- we sort the $x_k$ elements based on their objective value $f(x_k)$ from the smallest value up

The normal distribution defined by the covariance matrix can be written in many ways. We will use this to our advantage when implementing the algorithm: $N(m, C) ∼ m + C^{\frac{1}{2}}\cdot N(0,I)$
</br> The following equality also holds: $C = B D^{2} B^T$, where $B$ are the **eigenvectors** and $D$ is the **diagonal matrix** with **eigenvalues** of $C$ on the diagonal.
</br> The reason that the **eigendecoposition** surely exists is in the positive definite trait of the matrix $C$
</br> Through obvious matrix traits known from linear algebra we can see that $C^{\frac{1}{2}} = BD^{-1}B^T$

Now that we have the offsprings we have to calculate the new **mean**. This is the weighted average of $\mu$ selected points from our sample. ${m^{(g+1)}} = m^{(g)} + \sum_{i=1}^{\mu}{w_i \cdot x_{i:\lambda}^{(g+1)}}$
</br> The $w_i$ symbol represents the weights in our vector **``weights``** for recombination and $\mu$ is the parent population size **``parents``**.

**``parents_eff``** or ${\mu}_{eff}$ represents the **variance effective selection mass** for reasonble setting of weights. We also track 2 different **evolution paths**, which store information about the changes made to the mean vector in each generation of the algorithm. 

We express one as $p_c^g$ and it tells us the cumulative sum of all steps up to generation $g$. In practice we use exponential soothing for this step. 
</br>$p_{c}^{(g+1)} = (1 - c_{c}) p_{c}^{(g)} + h_{\sigma} \cdot \sqrt{c_{c}(2 - c_{c}){\mu}_{eff}} \cdot \frac{m^{(g+1)} - m^{(g)}}{\sigma}$

We also use **step size control** to regulate the overall scale of the distribution. For this the second evolution path is used, marked as $p_{\sigma}^g$. If the single steps cancel each other out, step size is too large and should be decreased. If the steps are pointing to a similar direction we can correct the step size to a larger number to cover these simlar steps in less steps. The optimal situation is having **uncorrelated steps**.
</br>$p_{\sigma}^{(g+1)} = (1 - c_{\sigma}) p_{\sigma}^{(g)} + \sqrt{c_{\sigma}(2 - c_{\sigma}){\mu}_{eff}} \cdot C^{(g)^{\frac{1}{2}}} \cdot \frac{m^{(g+1)} - m^{(g)}}{\sigma}$

Then we update the covariance matrix using this value.
</br>$C^{(g+1)} = (1 - \text{ learning rate} - c_{\mu}) \cdot C^{(g)} + \text{ learning rate} \cdot p_c \cdot p_c^T + c_{\mu} \sum_{i=1}^{\mu}{w_i \cdot \frac{x_{i:\lambda} - m}{\sigma} \cdot (\frac{x_{i:\lambda} - m}{\sigma})^T}$

$h_{\sigma}$ is the Heaviside function which stalls the update of the $p_c$ path in necessary circumstances.

With all this theoretical background, one can understand the algorithm bellow. First we set the parent and offspring population size, the weights step size control **``cs``**, covariance matrix adaptation **``cc``** and the **``learning_rate``**. The evolution paths **``ps``** and **``pc``** must be set to zero and **``C``** is set to I. Then we chose the starting mean **``mean``** and the step size **``sigma``**. At this point we start iterating the procedure until the termination condition is met. We then semple a new population of search points, do the selection and recombination, watch the setp size control and do the covariance matrix adaptation.

In [428]:
# implementation
def cma_es(limit, dim, fun, stopfitness):

    # initialization
    mean = np.random.rand(dim,1) # starting mean
    sigma = 0.3 # step size

    # selection parameters
    offsprings = 4 + math.floor(3 * math.log(dim)) # size of the offspring population 
    parents = offsprings / 2 # size of the parent population
    weights = np.log(parents + 1/2) - np.log(np.arange(1, math.floor(parents) + 1))
    weights = np.atleast_2d(weights / np.sum(weights)).T # normalization
    parents = math.floor(parents)
    parents_eff = np.sum(weights)**2 / np.sum(weights**2)
    

    # adaptation parameters
    cc = (4 + parents_eff / dim) / ((dim + 4) + (2 * parents_eff / dim)) 
    cs = (parents_eff + 2) / (dim + parents_eff + 5) 
    learning_rate = 2 / ((dim + 1.3)**2 + parents_eff)
    cmu_temp = 2 * (parents_eff - 2 + 1/parents_eff) / ((dim + 2)**2 + parents_eff)
    cmu = min(1 - learning_rate, cmu_temp)
    damps = 1 + 2 * max(0, math.sqrt((parents_eff - 1) / (dim + 1)) - 1) + cs

    # setting initial generation 0 paths and matrices
    pc = np.zeros((dim,1))
    ps = np.zeros((dim,1))
    B = np.eye(dim)
    D = np.ones((dim,1))
    C = np.matmul(np.matmul(B, np.diag((D**2).flatten())), B.T)
    invsqrtC = np.matmul(np.matmul(B, (np.diag((D**-1).flatten()))), B.T)
    eigeneval = 0
    chi_dim = dim**0.5 * (1 - 1/(4 * dim) + 1/(21* dim**2))

    # iterations
    function_calls = 0
    coordinates = np.zeros((dim, offsprings))
    value = np.zeros(offsprings)
    
    while(function_calls < limit):

        # generating and evaluating lambda offsprings
        for i in range(offsprings):
            coordinates[:, i] = \
                (mean + sigma * np.matmul(B, D * np.random.normal())).flatten()

            # checking constraints and repairing coordinates
            for k in np.nonzero([coordinates[:, i] > upper_bound]):
                coordinates[k, i] = upper_bound
            for k in np.nonzero([coordinates[:, i] < lower_bound]):
                coordinates[k, i] = lower_bound

            value[i] = fun(coordinates[:, i])
            function_calls = function_calls + 1
        
        # sort by values and compute weighted mean
        indices = np.argsort(value)
        value = np.sort(value)

        old_mean = mean
        mean = np.matmul(coordinates[:, indices[:parents]], weights)
    
        # update evolution paths
        ps = (1 - cs) * ps + np.sqrt(cs * (2 - cs) * parents_eff) 
        ps = ps * np.matmul(invsqrtC, ((mean - old_mean)) / sigma)
        cond = np.sqrt(1 - (1 - cs)**(2 * function_calls / offsprings)) / chi_dim
        cond = np.linalg.norm(ps) / cond
        hsig = cond < 1.4 + 2/(dim + 1)
        pc = (1 - cc) * pc + hsig * np.sqrt(cc * (2 - cc) * parents_eff) 
        pc = pc * ((mean - old_mean) / sigma)
        
        # adapt covariance matrix C
        repeat_matrix = np.matlib.repmat(old_mean, 1, parents)
        temp = (1 / sigma) * (coordinates[:, indices[:parents]] - repeat_matrix) 
        C = (1 - learning_rate - cmu) * C 
        C = C + learning_rate * (np.matmul(pc, pc.T) + (1 - hsig) * cc * (2 - cc) * C)
        C = C + cmu * np.matmul(np.matmul(temp, np.diag(weights.flatten())), temp.T)

        # adapt step size sigma based on calculated ps
        sigma = sigma * math.exp((cs / damps) * (np.linalg.norm(ps) / chi_dim - 1)); 
        
        # decomposition of C for the next iteration
        if function_calls - eigeneval > offsprings / (learning_rate + cmu) / dim /10:
            eigeneval = function_calls # time optimization
            C = np.triu(C,0) + np.triu(C,1).T # for symmetry
            D, B = np.linalg.eig(C) 
            D = np.real(D) # to avert complex matrix warning
            B = np.real(B)
            D = np.sqrt(D)
            invsqrtC = np.matmul(np.matmul(B, (np.diag(D**-1))), B.T)
            D = np.atleast_2d(D).T
        
        # termination condition
        if value[0] <= stopfitness + 0.01:
            break

    return coordinates[:, indices[0]], fun(coordinates[:, indices[0]])[0]


In [387]:
# testing our algorithm
min_values_3 = [float('inf') for i in range(len(functions))]
min_coordinates_3 = [[0 for i in range(40)] for k in range(len(functions))]
no_calls = 10

for k in range(no_calls):
    for i in range(len(functions)):
        curr_coordinates, curr_value = \
            cma_es(100000, 40, functions[i+1], true_minimums[i+1])
        if curr_value < min_values_3[i]:
                min_values_3[i] = curr_value
                min_coordinates_3[i] = curr_coordinates.copy()

comparison(min_values_3)
results_to_file(min_coordinates_3, 3)

1                    TRUE: 21.1                 APPROXIMATE: 21.11                DIFFERENCE: 0.01
2                    TRUE: 26.91                APPROXIMATE: 1063.63              DIFFERENCE: 1036.72
3                    TRUE: 311.6                APPROXIMATE: 445.92               DIFFERENCE: 134.32
4                    TRUE: 311.6                APPROXIMATE: 694.65               DIFFERENCE: 383.05
5                    TRUE: -48.47               APPROXIMATE: -40.03               DIFFERENCE: 8.44
6                    TRUE: -91.36               APPROXIMATE: -90.45               DIFFERENCE: 0.91
7                    TRUE: 32.49                APPROXIMATE: 99.46                DIFFERENCE: 66.97
8                    TRUE: 71.6                 APPROXIMATE: 100.35               DIFFERENCE: 28.75
9                    TRUE: -356.7               APPROXIMATE: -349.91              DIFFERENCE: 6.79
10                   TRUE: 51.03                APPROXIMATE: 1058.87              DIFFERENCE: 1007.8

## **ALGORITHM 4:** Nelder-Mead method
This fourth alogirthm is a numerical method used to find the minimum of an objective function, in our case BBOB functions, in a multidimensional space.
The method uses the concept of a **simplex**, which is a special **polytope** *(a geometric object with flat sides)* of $n+1$ vertices in n dimensions *(for example on a plane this would be a triangle)*. 
</br>Now for the quick summary of how this algorithm works. As the initial point we take a point of all **zeros** and generate others with a fixed step along each dimension in turn. Then we start the simplex iteration. We are trying to minimize function $f(x)$, where $x \in \mathbb{R}$. Our current points are $ x_1, …, x_{n+1}$.

1. **ORDER:**  Order according to the values at the vertices: $f(x_1) \leq … \leq f(x_{n+1})$ and check if the method should stop.

2. **CENTROID:** Calculate the centroid $x_0$ of all points except $x_{n+1}$.

3. **REFLECTION:** Compute reflected point $x_r = x_0 + \alpha(x_0 – x_{n+1})$, for $\alpha = 1$. 
</br>If the reflected point is better than the second worst, but not better than the best, $f(x_1) \leq f(x_r) \lt f(x_n)$, 
</br>than obtain a new simplex by replacing the worst point $x_{n+1}$ with the reflected point $x_r$, and go to step 1.

4. **EXPANSION:** If the reflected point is the best point so far, $f(x_r) \lt f(x_1)$, then compute the expanded point $x_e = x_0 + \gamma(x_r – x_0)$ with $\gamma = 2$. 
</br>If the expanded point is better than the reflected point, $f(x_e) \lt f(x_r)$, then obtain a new simplex by replacing the worst $x_{n+1}$ with the expanded point $x_e$ and go to step 1; 
</br>else obtain a new simplex by replacing the worst point $x_{n+1}$ with the reflected point $x_r$ and go to step 1.

5. **CONTRACTION:** Here it is certain that $f(x_r) \geq f(x_n)$, where $x_n$ is second to the worst point. 
</br>
</br>If $f(x_r) \lt f(x_{n+1})$, then compute the contracted point on the outside $x_c = x_0 + \rho(x_r – x_0)$ with $\rho=0.5$. 
</br>If the contracted point is better than the reflected point, $f(x_c) \lt f(x_r)$, then obtain a new simplex by replacing the worst point $x_{n+1}$ with the contracted point $x_c$ and go to step 1.
</br>Else go to step 6. 
</br>
</br>If $f(x_r) \geq f(x_n+1)$, then compute the contracted point on the inside $x_c = x_0 + \rho*(x_{n+1} – x_0)$ with $\rho=0.5$. 
</br>If the contracted point is better than the worst point, $f(x_c) \lt f(x_{n+1})$, then obtain a new simplex by replacing the worst point xn+1 with the contracted point $x_c$ and go to step 1. 
</br>Else go to step 6.

6. **SHRINK:** Replace all points except the best ($x_1$) with $x_i = x_1 + \sigma(x_i – x_1)$ and go to step 1.


In [414]:
# implementation
def nelder_mead(limit, dim, f, tolerance=10e-9, how_many_times_same_result=500, step=1):
    best_coordinates = np.zeros(dim)
    best_value = f(best_coordinates)

    curr_result = []
    curr_result.append([best_value, best_coordinates])

    for i in range(dim):
        curr = best_coordinates.copy()
        curr[i] = curr[i] + step
        curr_result.append([f(curr), curr])

    for k in range(limit):
        # order
        curr_result.sort(key=lambda x: x[0])
        best = curr_result[0][0]

        if best >= best_value - tolerance:
            how_many_times_same_result -= 1
        else:
            how_many_times_same_result = 500
            best_value = best

        if (how_many_times_same_result <= 0): 
            return curr_result[0][1], curr_result[0][0][0]

        # centroid
        cent = centroid(dim, curr_result)

        # reflection
        ref, check1 = reflection(cent, curr_result, f)
        if (check1): continue

        # expansion
        check2 = expansion(cent, ref, curr_result, f)
        if (check2): continue

        # contraction
        check3 = contraction(cent, ref, curr_result, f)
        if (check3): continue

        # shrink
        curr_result = shrink(curr_result, f)
        curr_result[0][1] = constraints(curr_result[0][1])

    return curr_result[0][1], curr_result[0][0][0]
    
def centroid(dim, curr_result):
    cent = np.zeros(dim)
    for el in curr_result[:-1]:
        for index, number in enumerate(el[1]):
            cent[index] = cent[index] + number / (len(curr_result)-1)
    return cent

def reflection(c, curr_result, f, alpha=1):
    r = c + alpha*(c - curr_result[-1][1])
    value_r = f(r)
    check = False

    if curr_result[0][0] <= value_r < curr_result[-2][0]:
        del curr_result[-1]
        curr_result.append([value_r, r])
        check = True

    return r, check


def expansion(c, r, curr_result, f, gamma=2):
    value_r = f(r)
    check = False

    if value_r < curr_result[0][0]:
        e = c + gamma*(r - c)
        value_e = f(e)

        if value_e < value_r:
            del curr_result[-1]
            curr_result.append([value_e, e])
        elif value_e >= value_r:
            del curr_result[-1]
            curr_result.append([value_r, r])
        check = True

    return check


def contraction(c, r, curr_result, f, rho=0.5):
    check = False
    value_r = f(r)

    if value_r < curr_result[-1][0]:
        cont = c + rho*(r - c)
        value_cont = f(cont)
        if value_cont < value_r:
            del curr_result[-1]
            curr_result.append([value_cont, cont])
            check = True
    else:
        cont = c + rho*(curr_result[-1][1] - c)
        value_cont = f(cont)
        if value_cont < curr_result[-1][0]:
            del curr_result[-1]
            curr_result.append([value_cont, cont])
            check = True

    return check


def shrink(curr_result, f, sigma=0.5):
    result = []

    for r in curr_result:
        x = curr_result[0][1] + sigma*(r[1] - curr_result[0][1])
        result.append([f(x), x])

    return result

def constraints(coordinates):
    for k in np.nonzero([coordinates > upper_bound]):
            coordinates[k] = upper_bound
    for k in np.nonzero([coordinates < lower_bound]):
        coordinates[k] = lower_bound
    return coordinates


In [420]:
# testing the code
min_values_4 = [float('inf') for i in range(len(functions))]
min_coordinates_4 = [[0 for i in range(40)] for k in range(len(functions))]
no_runs = 1 

dim = 40
limit = 1000000

for i in range(no_runs):
    for k in range(len(functions)):
        curr_coordinates, curr_value = nelder_mead(limit, dim, functions[k+1])
        if curr_value < min_values_4[k]:
            min_values_4[k] = curr_value
            min_coordinates_4[k] = curr_coordinates.copy()
    
comparison(min_values_4)
results_to_file(min_coordinates_4, 4)

1                    TRUE: 21.1                 APPROXIMATE: 21.1                 DIFFERENCE: 0.0
2                    TRUE: 26.91                APPROXIMATE: 21727.95             DIFFERENCE: 21701.04
3                    TRUE: 311.6                APPROXIMATE: 1157.3               DIFFERENCE: 845.7
4                    TRUE: 311.6                APPROXIMATE: 846.91               DIFFERENCE: 535.31
5                    TRUE: -48.47               APPROXIMATE: -48.47               DIFFERENCE: 0.0
6                    TRUE: -91.36               APPROXIMATE: 92.68                DIFFERENCE: 184.04
7                    TRUE: 32.49                APPROXIMATE: 189.3                DIFFERENCE: 156.81
8                    TRUE: 71.6                 APPROXIMATE: 145.33               DIFFERENCE: 73.73
9                    TRUE: -356.7               APPROXIMATE: -324.23              DIFFERENCE: 32.47
10                   TRUE: 51.03                APPROXIMATE: 2583.39              DIFFERENCE: 2532

### **Results**

In conclusion the best results overall were with the **CMA-ES** algorithm but the last one did pretty good as well. Finally let us present the solutions. 

| Algorithm           | Function 1 | Function 2 | Function 3 | Function 4 | Function 5 |
|---------------------|------------|------------|------------|------------|------------|
| Global minimum      | 21.1       | 26.91      | 311.6      | 311.6      | -48.47     |
| Gradient Descent    | 21.1       | 39815.63   | 1441.91    | 1747.75    | -23.91     |
| Simulated annealing | 21.39      | 25920.63   | 1256.07    | 1330.9     | 168.67     | 
| CMA-ES              | 21.11      | 1063.63    | 445.92     | 694.65     | -40.03     | 
| Nelder-Mead         | 21.1       | 21727.95   | 1157.3     | 846.91     | -48.47     | 

</br>

| Algorithm           | Function 6 | Function 7| Function 8 | Function 9 | Function 10 | 
|---------------------|------------|-----------|------------|------------|-------------|
| Global minimum      | -91.36     | 32.49     | 71.6       | -356.7     | 51.03       |
| Gradient Descent    | -29.92     | 2225.85   | 499.89     | -120.22    | 139871.12   |
| Simulated annealing | -67.81     | 433.45    | 122.27     | -310.42    | 26798.82    |
| CMA-ES              | -90.45     | 99.46     | 100.35     | -349.91    | 1058.87     |
| Nelder-Mead         | 92.68      | 189.3     | 145.33     | -324.23    | 2583.39     |

</br>

| Algorithm           |Function 11  |Function 12 |Function 13 |Function 14 |Function 15 |
|---------------------|------------ |------------|------------|------------|------------|
| Global minimum      | -96.65      | 553.39      | 9.88        | 405.47      | 64.25       |
| Gradient Descent    | 341.97      | 728246.51  | 288.43     | 405.49     | 575.29     |
| Simulated annealing | 160.02      | 76399.54   | 74.85      | 405.69     | 1246.38    |
| CMA-ES              | 32.13       | 553.47     | 9.9        | 405.48     | 206.53     |
| Nelder-Mead         | -16.14      | 554.76     | 18.3       | 405.48     | 1169.91    |

</br>

| Algorithm           | Function 16 | Function 17 | Function 18 | Function 19 | Function 20 |
|---------------------|-------------|-------------|-------------|-------------|-------------|
| Global minimum      | -43.28      | 227.51      | 227.51      | 73.06       | -123.81     |
| Gradient Descent    | -9.14       | 229.34      | 253.28      | 90.73       | -122.66     |
| Simulated annealing | -31.87      | 238         | 262.26      | 91.88       | -121.81     |
| CMA-ES              | -34.93      | 229.88      | 232.09      | 73.99       | -122.25     |
| Nelder-Mead         | -17.1       | 241.31      | 281.04      | 73.08       | -121.28     |

</br>

| Algorithm           | Function 21 | Function 22 | Function 23 | Function 24 |
|---------------------|-------------|-------------|-------------|-------------|
| Global minimum      | -44.42      | 222.1       | -1000       | -1.33       |
| Gradient Descent    | -43.72      | 224.16      | -995.72     | 840.97      |
| Simulated annealing | -44.3       | 224.12      | -997.65     | 1041.31     |
| CMA-ES              | -38.17      | 224.69      | -998.56     | 58.71       |
| Nelder-Mead         | -38.17      | 224.69      | -997.62     | 240.05      |

<br/>